In [1]:
!pip install -q datasets
!pip install -q bitsandbytes
!pip install -q peft
!pip install -q accelerate
!pip install -q trl
!pip install -q wandb
!pip install -q -U google-generativeai
!pip install -q openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 10

In [2]:
import torch
from datasets import load_dataset
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

from huggingface_hub import notebook_login
notebook_login()

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "Pennlaine/Mistral-7B-v02-Entity-Extraction"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

In [4]:
prompt = '''<s>[INST]John Doe, a 45-year-old male, presents with a two-week history of progressive chest pain and shortness of breath. The chest pain is described as a tight, squeezing sensation located centrally and radiating to the left arm and jaw. It is aggravated by physical exertion and alleviated by rest. The patient reports associated symptoms of palpitations, diaphoresis, and nausea. He has a history of hypertension, hyperlipidemia, and type 2 diabetes mellitus. His family history is significant for myocardial infarction in his father at age 60. The patient has a 20-pack-year smoking history but quit 5 years ago. He occasionally consumes alcohol and leads a sedentary lifestyle with a diet high in processed foods and red meat. Current medications include metformin, lisinopril, atorvastatin, and aspirin.
Extract the name, age, symptoms, medical history, family history, father death age, medication history  and return in Json format.[/INST]```json'''

prompt = '''<s>[INST] Your task is to extract entities as mentioned in the clinical text.
consider the example, where given a clinical text, entities are extracted as follows.

clinical text: Clinic date 25/09/2018  Re Mr Owen Owens30/06/1968  Dear doctor,  Diagnosis: Symptomatic structural focal epilepsy Area of ischaemic damage left inferior frontal lobe Significant anxiety and depression  I reviewed this 50 year old man, together with his wife, in clinic today. He was well from an epilepsy point of view until he had a   seizur  e last month. His wife heard a bang from the next room and went in to see him unconsciouss on the floor. He was stiff, his eyes were open and he made unusual groaning noises before starting to shake. This lasted about 40 seconds before he went to a “deep sleep”. It took him several hours to recover fully. Mr Owens himself remembers his right leg twitching before he lost consciousness. I think therefore that this was a   focal to bilateral convulsive   seizur    es. As you recall Mr Owens has had focal motor   seizur  es in the past where he has had jerking of his right leg. These were happening frequently before he started the medication. The last event was probably 10 months ago. He has had one previous   focal to bilateral convulsive   seizur    e at the time of diagnosis of his epilepsy in May 2017. As you will recall his MRI at the time was abnormal with an area of ischaemic damage in the left inferior frontal lobe. This is the probable cause of his epilepsy. An EEG in 2017 did show some left sided sharp waves.    He is currently taking lamotrigine 75mg twice a day   and given the recent   seizur  e I would be very grateful if you could increase this by 25mg every fortnight to 100mg twice a day. It can be increased further if necessary in a similar way. Previously he tried levetiracetam but this made his mood worse.  Mr Owens has ongoing problems with his mood and finds it very difficult to leave the house. He has found anti-depressants useful in the past and I would be very grateful if you could reassess him from his mental health point of view. There is no contraindication to treatment with anti-depressants if necessary. Mr Owens is still managing to work as an accountant and he is aware of the DVLA driving regulations. He is currently drinking around 40units of alcohol a week. We had a chat about the effects of alcohol and I would advise that he reduces his intake to the recommended 14 units of alcohol per week with a few alcohol free days. I will ask the epilepsy nurses to review again. Yours.
E```json'''
max_new_tokens = 100
inputs = tokenizer.encode(prompt, return_tensors='pt')
outputs = model.generate(inputs, max_new_tokens=max_new_tokens, num_return_sequences=1)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[INST] Your task is to extract entities as mentioned in the clinical text.
consider the example, where given a clinical text, entities are extracted as follows.

clinical text: Clinic date 25/09/2018  Re Mr Owen Owens30/06/1968  Dear doctor,  Diagnosis: Symptomatic structural focal epilepsy Area of ischaemic damage left inferior frontal lobe Significant anxiety and depression  I reviewed this 50 year old man, together with his wife, in clinic today. He was well from an epilepsy point of view until he had a   seizur  e last month. His wife heard a bang from the next room and went in to see him unconsciouss on the floor. He was stiff, his eyes were open and he made unusual groaning noises before starting to shake. This lasted about 40 seconds before he went to a “deep sleep”. It took him several hours to recover fully. Mr Owens himself remembers his right leg twitching before he lost consciousness. I think therefore that this was a   focal to bilateral convulsive   seizur    es. As you r

In [ ]:
!export OPENAI_API_KEY=""

In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [8]:
import os

In [18]:
from openai import OpenAI
client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "write a haiku about ai"}
    ]
)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}